## Permutation

The permutation optime is base on the the Permutation Primitive Operation (PPO) is defined as:
$\begin{equation}
    PPO(R,p,m)=  (R \ggg  p) \land m \tag{1}
\end{equation}$
In this equation, $R$ represents the operation register on the bitsliced representation, $\ggg$ denotes the right cyclic shift operation, $p$ is the number of shifts, $\land$ is the bitwise AND operation, and $m$ is the mask.

In [1]:
# let use the tau permutation to do the example for explanation.
tau = [0, 11, 6, 13, 10, 1, 12, 7, 5, 14, 3, 8, 15, 4, 9, 2]


# the right cyclic shift tau
def right_cyclic_shift(tau: list):

    shift = []
    for i in range(len(tau)):
        if tau.index(i) >= i:
            shift.append(tau.index(i) - i)
        else:
            shift.append(len(tau) - i + tau.index(i))
    return shift


tau_shift = right_cyclic_shift(tau)
print(tau_shift)
ps = [(x, i) for i, x in enumerate(tau_shift)]
print(ps)

[0, 4, 13, 7, 9, 3, 12, 0, 3, 5, 10, 6, 10, 6, 11, 13]
[(0, 0), (4, 1), (13, 2), (7, 3), (9, 4), (3, 5), (12, 6), (0, 7), (3, 8), (5, 9), (10, 10), (6, 11), (10, 12), (6, 13), (11, 14), (13, 15)]


## merge

Given $m_1$ and $m_2$ as the masks, the following holds:
$\begin{equation}
    PPO(R,p,m_1) \lor PPO(R,p,m_2) = PPO(R,p,m_1 \lor m_2)\tag{2}
\end{equation}$
In this equation, the symbol $\lor$ denotes the bitwise OR operation.

In [2]:
# This code snippet is used to identify certain values within the `tau_shift` list. It then records the indices of these values into a separate list. Finally, it pairs each index with its corresponding value from the `tau_shift` list.


def merge_shift_ppo(tau_shift: list) -> list:
    shift_ppo = []
    # Remove duplicate elements for `tau_shift`
    tau_shift_set = list(set(tau_shift))

    for shift in tau_shift_set:
        merge_shift = [i for i, x in enumerate(tau_shift) if x == shift]
        shift_ppo.append((shift, merge_shift))
    return shift_ppo


merge_shift_ppo_result = merge_shift_ppo(tau_shift)
print(merge_shift_ppo_result)

[(0, [0, 7]), (3, [5, 8]), (4, [1]), (5, [9]), (6, [11, 13]), (7, [3]), (9, [4]), (10, [10, 12]), (11, [14]), (12, [6]), (13, [2, 15])]


## spilt

 Let $p$, $p_1$, and $p_2$ be the number of shifts, and let $m_1 = m \lll p_2$, where $p=p_1+p_2$ and $\lll$ represents the left cyclic shift. Then, the following equation holds:
$\begin{equation}
    PPO(R,p ,m) = PPO(PPO(R,p_1,m_1),p_2,m) \tag{3}
\end{equation}$

In [35]:
# preprocess the merge_shift_ppo_result by the dictionary
merge_shift_ppo_result.reverse()
merge_ppo = {t[0]: t[1] for t in merge_shift_ppo_result}
merge_ppo.pop(0)  # remove the 0 shift
print(merge_ppo)

{13: [2, 15], 12: [6], 11: [14], 10: [10, 12], 9: [4], 7: [3], 6: [11, 13], 5: [9], 4: [1], 3: [5, 8]}


In [54]:
def split_ppo(split: dict, no_split: dict) -> dict:
    if len(no_split) == 0:
        return split
    temp_no_split = dict(no_split)
    temp_split = dict(split)
    # no sure split the element is mean goal optimal, so for traversal the element on split and no_split
    shift, positions = temp_no_split.popitem()

    # not split the element
    no_split_list = split_ppo({shift: positions, **temp_split}, {**temp_no_split})
    split_list = no_split_list
    # split the element
    # 1. get the keys of the temp_split
    shifts = list(temp_split.keys())
    shifts.reverse()
    # 2. let the shift split to the tow element sum of the shifts, the shifts is increasing order
    for i in range(0, len(shifts) - 1):
        for j in range(i + 1, len(shifts)):
            if shifts[i] + shifts[j] == shift:
                # split the element
                temp_split[shifts[i]] = temp_split[shifts[i]] + positions
                temp_split[shifts[j]] = temp_split[shifts[j]] + positions
                split_list = split_ppo({**temp_split}, {**temp_no_split})
    return no_split_list if len(no_split_list) < len(split_list) else split_list


split_ppo_result = {}
split = split_ppo(split_ppo_result, merge_ppo)
print(split)

{9: [4, 6, 2, 15], 6: [11, 13, 10, 12, 14], 5: [9, 14], 4: [1, 3, 10, 12, 2, 15], 3: [5, 8, 3, 6]}


In [57]:
# transform the split result to the list
split_list = []
for shift, positions in split.items():
    split_list.append((shift, positions))
split_list.reverse()
print(split_list)

[(3, [5, 8, 3, 6]), (4, [1, 3, 10, 12, 2, 15]), (5, [9, 14]), (6, [11, 13, 10, 12, 14]), (9, [4, 6, 2, 15])]
